# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import re
from collections import Counter


def words(text):
    return re.findall(r"\w+", text.lower())


WORDS = Counter(words(open("data/big.txt", encoding="unicode_escape").read()))


def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N


def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)


def candidates_with_prob(word):
    "Generate possible spelling corrections for word."
    return [
        (c, P(c))
        for c in (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    ]


def candidates(word):
    "Generate possible spelling corrections for word."
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = "abcdefghijklmnopqrstuvwxyz"
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [2]:
# quick test
candidates_with_prob("dking")

[('doing', 0.00015872562937387197),
 ('king', 0.00021133693349217785),
 ('dying', 7.133736151634695e-05)]

## My implementation

The main problem with Norvig's solution, is that it does not take context words in an account. So this is simple greedy approach, which works, but not that good. We should expect accuracy of bigram model be at least as good, as Norvig's solution, and five-gram model, to outperform them both.

### Store all context
My first idea was to generate all variations of context, generate all variations of next word (which we predicted to be correct) and store them in model. So, e.g. if my bigram dataset is 

```csv
4 what is
6 the strongest
```

then, I would store a map like this:

```python
map["the"]["strongest"] = 6
map["thea"]["strongest"] = 6
map["theb"]["strongest"] = 6
map["thec"]["strongest"] = 6
# ... all possible context and word edits
```

The problem with this approach, is there are simply too much contexts to generate and store, which results in a very long computation times. I dropped this idea immediately.

### Generate context during inference

Second idea was to do basically the same, but iterate over all contexts and next words during inference. And I wrote code for this approach

In [3]:
import pandas as pd

# keep_default_na in order for "null" to be converted to None
bigram = pd.read_csv("data/w2.txt", sep="\t", header=None, keep_default_na=False)
fivegram = pd.read_csv("data/w5.txt", sep="\t", header=None, keep_default_na=False)
bigram.tail()

,0,1,2
1020363,24,zviad,gamsakhurdia
1020364,25,zweimal,leben
1020365,24,zwick,and
1020366,24,zydeco,music
1020367,72,zz,top


In [4]:
bigram.isna().sum().sum(), fivegram.isna().sum().sum()

(0, 0)

In [7]:
from itertools import product
from functools import cache
from collections import defaultdict
from tqdm import tqdm


@cache
def get_edits(word: str):
    if len(word) <= 2:
        return {word}

    return {word} | edits1(word)


class NGramModel:
    def __init__(self, df: pd.DataFrame):
        # Create a placeholder for model
        model = defaultdict(lambda: defaultdict(lambda: 0))

        # Count frequency of co-occurance
        t = tqdm(total=df.shape[0])
        for _, ngram in df.iterrows():
            t.update()
            count, *words = ngram
            *context, word = words

            model[tuple(sorted(context))][word] += count

        # Let's transform the counts to probabilities
        for context in model:
            total_count = float(sum(model[context].values()))
            for suggestion in model[context]:
                model[context][suggestion] /= total_count

        self.model = model
        self.context_len = len(context)

    def suggest(self, context: list[str], word: str) -> str:
        best_suggestion = None
        best_prob = 0

        # if context is too short, we can't make any suggestions
        context = [item if item else "<UNK>" for item in context]
        context = context[-self.context_len :]

        # get all possible contexts
        all_contexts = lambda: product(*[get_edits(c) for c in sorted(context)])

        # if the word exists, then it is very probable to be correct
        for probable_context in all_contexts():
            prob = self.model[tuple(probable_context)][word]
            if 0.95 * prob > best_prob:
                best_prob = 0.95 * prob

        # but, it might happen, that changes of this word might be more probable
        for probable_context in all_contexts():
            for suggestion in get_edits(word):
                prob = self.model[tuple(probable_context)][suggestion]
                if 0.05 * prob > best_prob:
                    best_prob = 0.05 * prob
                    best_suggestion = suggestion

        if best_suggestion is None:
            return sorted(candidates_with_prob(word), key=lambda x: x[1])[-1]

        return best_suggestion, best_prob

In [8]:
model = NGramModel(bigram)
# model.model = d

  0%|          | 0/1020368 [00:00<?, ?it/s]

100%|██████████| 1020368/1020368 [00:34<00:00, 29270.49it/s]


In [9]:
# random context uses Norvig's suggestion
print(model.suggest([None], "fich"))

print(model.suggest(["good"], "maan"))
print(model.suggest(["defalt"], "vales"))

print(model.suggest(["psychologyst"], "too"))
print(model.suggest(["too"], "psychologyst"))

print(model.suggest(["sport"], "dking"))
print(model.suggest(["species"], "dking"))

('rich', 8.2037965743799e-05)
('man', 0.0002760255055068708)
('values', 0.0011465603190428714)
('to', 0.0016368481157213552)
('psychologist', 1.6330975460692016e-07)
('king', 0.00021133693349217785)
('king', 0.00021133693349217785)


Unfortunately, because there are no bigrams `doing sport` or `dying species` in the `w2.txt` dataset, the model cannot correct them.

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I tried testing with both `w2.txt` and `w5.txt` dataset, and, due to the exponential growth of `probable_context` with the number of ngrams, inference for `w5.txt` takes simply too much time. That is why I didn't use it. This is because this algorithm is not very efficient.

You can clearly see, that I am using edit distance with weights of `0.95` for word without correction and `0.05` for word with 1 correction. This is because if the word exists in the dataset, then it is very likely, that this word is correct. However, it might happend, that this word does not exist in dataset or the correction will have hight probability to occur.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

I decided to use `github-corpus` dataset, because I found it very easy to work with

In [10]:
gh_df = pd.read_json("data/github-corpus.json", lines=True)
gh_df.head()

,repo,commit,message,edits
0,https://github.com/abacritt/angularx-social-login,d4c912f5ccd70c81f424fadbf1fe1a2ecb942f07,Fix typo\n,[{'src': {'text': ' IN.User.authori...
1,https://github.com/abacritt/angularx-social-login,8beb5a5ebee0882142541dc84c004f6ce3165f94,fix typo\n\nfix typo in firstname\n,[{'src': {'text': ' user.em...
2,https://github.com/abahmed/Deer,44781b8842c7e647d1f5d2417d21244e815c5eec,fixed typo (#263)\n\n,[{'src': {'text': ' :de: | Deutsch...
3,https://github.com/abakan/ablog,1cee106975e3137cb9a667729e832cc9494f0692,Fixed typo.\n,"[{'src': {'text': ' :nocoments:', 'path..."
4,https://github.com/abakan/ablog,4e11caf1f7ebe611ffb08f8a6909ac6752d784cd,Fixed typo\n,[{'src': {'text': ' You can disable comments...


In [27]:
def generate_test_data(amount: int = 200):
    test_data = []
    for _, item in gh_df.iterrows():
        for test_case in item["edits"]:
            if not "is_typo" in test_case:
                continue
            if test_case["is_typo"] and test_case["src"]["path"].endswith(".md"):
                src = words(test_case["src"]["text"])
                tgt = words(test_case["tgt"]["text"])

                if len(src) != len(tgt) or src == tgt:
                    continue

                indecies = [i for i in range(len(src)) if src[i] != tgt[i]]
                for index in indecies:
                    test_data.append((src, tgt, index))
                    if len(test_data) >= amount:
                        break

    return test_data

In [28]:
test_data = generate_test_data()
print(*["{}\n{}\n{}\n".format(*test_case) for test_case in test_data][:5], sep="\n")

['de', 'deutsche']
['de', 'deutsch']
1

['add', 'watchdog', '0', 'to', 'disablle', 'watchdog', 'timer', 'if', 'you', 'get', 'accidental', 'reboots']
['add', 'watchdog', '0', 'to', 'disable', 'watchdog', 'timer', 'if', 'you', 'get', 'accidental', 'reboots']
4

['generates', 'a', 'random', 'hecadecimal', 'color', 'code']
['generates', 'a', 'random', 'hexadecimal', 'color', 'code']
3

['takes', 'a', 'veriadic', 'function', 'and', 'returns', 'a', 'closure', 'that', 'accepts', 'an', 'array', 'of', 'arguments', 'to', 'map', 'to', 'the', 'inputs', 'of', 'the', 'function']
['takes', 'a', 'variadic', 'function', 'and', 'returns', 'a', 'closure', 'that', 'accepts', 'an', 'array', 'of', 'arguments', 'to', 'map', 'to', 'the', 'inputs', 'of', 'the', 'function']
2

['checks', 'if', 'the', 'provided', 'intiger', 'is', 'primer', 'number']
['checks', 'if', 'the', 'provided', 'intiger', 'is', 'prime', 'number']
6



In [29]:
def evaluate_my_solution(model, test_data):
    correct = 0
    total = 0

    for src, tgt, index in tqdm(test_data):
        word = src[index]
        corr = tgt[index]

        pred, _ = model.suggest(src[:index], word)

        if pred == corr:
            correct += 1

        total += 1

    return correct / total


def evaluate_norvigs_solution(test_data):
    correct = 0
    total = 0

    for src, tgt, index in tqdm(test_data):
        word = src[index]
        corr = tgt[index]

        pred = sorted(candidates_with_prob(word), key=lambda x: x[1])[-1][0]

        if pred == corr:
            correct += 1

        total += 1

    return correct / total

In [30]:
evaluate_my_solution(model, test_data)

  0%|          | 0/201 [00:00<?, ?it/s]

100%|██████████| 201/201 [00:11<00:00, 16.88it/s]


0.39800995024875624

In [31]:
evaluate_norvigs_solution(test_data)

100%|██████████| 201/201 [00:06<00:00, 29.09it/s]


0.3383084577114428

## Conclusion

As you can see, bigram solution using my implementation outperforms the Norvig's solution on github dataset. It is a bit slower and with increasing context it will require exponentially more time to compute, but this is the cost of accuracy in my implementation.